In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import optuna

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, f1_score

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

In [ ]:
df = pd.read_csv('../data/data.csv')
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [43]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=True, random_state=0
)

In [44]:
scaler = StandardScaler()

X_train_original = X_train
X_test_original = X_test

X_train = X_train[['contrast_score', 'sharpness_score', 'noise_score']]
X_test = X_test[['contrast_score', 'sharpness_score', 'noise_score']]

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [26]:
def objective(trial):
    classifier_name = trial.suggest_categorical("classifier", [
        "LogisticRegression", "KNN", "DecisionTree", "RandomForest", "MLP", "SVC"
    ])

    if classifier_name == "LogisticRegression":
        C = trial.suggest_loguniform("lr_C", 1e-4, 1e2)
        model = LogisticRegression(C=C, max_iter=1000)

    elif classifier_name == "KNN":
        n_neighbors = trial.suggest_int("knn_n_neighbors", 1, 30)
        model = KNeighborsClassifier(n_neighbors=n_neighbors)

    elif classifier_name == "DecisionTree":
        max_depth = trial.suggest_int("dt_max_depth", 1, 20)
        model = DecisionTreeClassifier(max_depth=max_depth)

    elif classifier_name == "RandomForest":
        n_estimators = trial.suggest_int("rf_n_estimators", 10, 200)
        max_depth = trial.suggest_int("rf_max_depth", 2, 20)
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)

    elif classifier_name == "MLP":
        hidden_layer_sizes = trial.suggest_categorical("mlp_hidden_layer_sizes", [(50,), (100,), (50, 50)])
        alpha = trial.suggest_loguniform("mlp_alpha", 1e-5, 1e-1)
        model = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, alpha=alpha, max_iter=1000)

    elif classifier_name == "SVC":
        C = trial.suggest_loguniform("svc_C", 1e-2, 1e2)
        gamma = trial.suggest_loguniform("svc_gamma", 1e-4, 1e-1)
        model = SVC(C=C, gamma=gamma)

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    return acc


In [27]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

[I 2025-07-28 18:19:16,851] A new study created in memory with name: no-name-20f1d354-5695-4615-8a91-8f3fb6118836
/tmp/ipykernel_1176/229894683.py:29: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform("svc_C", 1e-2, 1e2)
/tmp/ipykernel_1176/229894683.py:30: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  gamma = trial.suggest_loguniform("svc_gamma", 1e-4, 1e-1)
[I 2025-07-28 18:19:16,949] Trial 0 finished with value: 0.490625 and parameters: {'classifier': 'SVC', 'svc_C': 0.1359492885699183, 'svc_gamma': 0.00047030382639870304}. Best is trial 0 with value: 0.490625.
/tmp/ipykernel_1176/229894683.py:29: FutureWarning: suggest_loguniform has been de

In [29]:
def get_best_model(params):
    name = params['classifier']
    if name == "LogisticRegression":
        return LogisticRegression(C=params['lr_C'], max_iter=1000)
    elif name == "KNN":
        return KNeighborsClassifier(n_neighbors=params['knn_n_neighbors'])
    elif name == "DecisionTree":
        return DecisionTreeClassifier(max_depth=params['dt_max_depth'])
    elif name == "RandomForest":
        return RandomForestClassifier(n_estimators=params['rf_n_estimators'], max_depth=params['rf_max_depth'])
    elif name == "MLP":
        return MLPClassifier(hidden_layer_sizes=params['mlp_hidden_layer_sizes'], alpha=params['mlp_alpha'], max_iter=1000)
    elif name == "SVC":
        return SVC(C=params['svc_C'], gamma=params['svc_gamma'])

In [30]:
best_model = get_best_model(study.best_params)
best_model.fit(X_train, y_train)
final_y_pred = best_model.predict(X_test)
print("Acurácia final com melhor modelo:", accuracy_score(y_test, final_y_pred))
print("Precisão final com melhor modelo:", precision_score(y_test, final_y_pred))
print("Recall final com melhor modelo:", recall_score(y_test, final_y_pred))
print("F1 final com melhor modelo:", f1_score(y_test, final_y_pred))
print(f'{best_model}')

Acurácia final com melhor modelo: 0.940625
Precisão final com melhor modelo: 0.9675324675324676
Recall final com melhor modelo: 0.9141104294478528
F1 final com melhor modelo: 0.9400630914826499
MLPClassifier(alpha=0.00011741869923165701, hidden_layer_sizes=(50, 50),
              max_iter=1000)


In [31]:
best_model

,hidden_layer_sizes,"(50, ...)"
,activation,'relu'
,solver,'adam'
,alpha,0.00011741869923165701
,batch_size,'auto'
,learning_rate,'constant'
,learning_rate_init,0.001
,power_t,0.5
,max_iter,1000
,shuffle,True
,random_state,None


In [36]:
import cv2
from visao.ImageLoader import ImageLoader
from visao.ImageVectorizer import ImageVectorizer

loader = ImageLoader()
vec = ImageVectorizer()

kali = loader.load('../real/suica.png')
# kali = cv2.resize(kali, (512, 512))
kalivec = vec(kali)
print(best_model.predict(scaler.transform(np.array([kalivec]))))

[1]


/home/diego/miniconda3/envs/visao/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [81]:
y_good = final_y_pred[np.where(X_test_original['class'].values == 'good')]
y_low_contrast = final_y_pred[np.where(X_test_original['class'].values == 'low_contrast')]
y_blurry = final_y_pred[np.where(X_test_original['class'].values == 'blurry')]
y_noisy = final_y_pred[np.where(X_test_original['class'].values == 'noisy')]

print(f'Good: Acc: {accuracy_score(y_good, np.ones(y_good.shape[0]))}')
print(f'Low contrast: Acc: {accuracy_score(y_low_contrast, np.zeros(y_low_contrast.shape[0]))}')
print(f'Blurry: Acc: {accuracy_score(y_blurry, np.zeros(y_blurry.shape[0]))}')
print(f'Noisy: Acc: {accuracy_score(y_noisy, np.zeros(y_noisy.shape[0]))}')

Good: Acc: 0.9141104294478528
Low contrast: Acc: 0.9583333333333334
Blurry: Acc: 0.9622641509433962
Noisy: Acc: 0.9821428571428571


In [85]:
import pickle

with open('tabular_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)


with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)